# Bitcoin Market Sentiment vs. Hyperliquid Trader Performance

**Objective**  
Explore how Bitcoin market sentiment (Fear vs. Greed) relates to trader behaviour and performance on Hyperliquid.  
This analysis merges two datasets:

1. `fear_greed_index.csv` — daily sentiment index with classifications (Extreme Fear → Extreme Greed).  
2. `historical_data.csv` — raw trade-level data from Hyperliquid.

The notebook walks through data cleaning, exploratory data analysis (EDA), visualisations, and key insights that can inform smarter trading strategies.

> **Tip for reviewers**: Clone this repo, install the minimal requirements (`pandas`, `matplotlib`, `seaborn`), and run notebook cells top‑to‑bottom.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Set plotting style
plt.rcParams['figure.figsize'] = (10, 6)
sns.set_theme(style='whitegrid')


## 1. Load Data

In [ ]:
DATA_DIR = Path('.')

fear_path = DATA_DIR / 'fear_greed_index.csv'
trade_path = DATA_DIR / 'historical_data.csv'

fear_df = pd.read_csv(fear_path)
trade_df = pd.read_csv(trade_path)

display(fear_df.head())
display(trade_df.head())


## 2. Pre‑processing

In [ ]:
# --- Fear & Greed dataset ---
fear_df['date'] = pd.to_datetime(fear_df['date'])
# Categorise sentiment into three buckets
fear_df['sentiment_bucket'] = fear_df['classification'].replace({
    'Extreme Fear':'Fear', 'Fear':'Fear',
    'Extreme Greed':'Greed', 'Greed':'Greed'
}).fillna('Neutral')

# --- Trade dataset ---
# Convert `Timestamp IST` -> datetime (format: DD-MM-YYYY HH:MM)
trade_df['trade_dt'] = pd.to_datetime(trade_df['Timestamp IST'], format='%d-%m-%Y %H:%M')
trade_df['date'] = trade_df['trade_dt'].dt.normalize()   # midnight
# Clean numeric columns
trade_df['Closed PnL'] = pd.to_numeric(trade_df['Closed PnL'], errors='coerce').fillna(0)


## 3. Merge sentiment with trades

In [ ]:
merged = trade_df.merge(
    fear_df[['date', 'sentiment_bucket']],
    on='date',
    how='left'
)
merged['sentiment_bucket'] = merged['sentiment_bucket'].fillna('Unknown')
merged.head()


## 4. Aggregate daily trader metrics

In [ ]:
daily = (merged
         .groupby(['date','sentiment_bucket'])
         .agg(trade_count=('Closed PnL','size'),
              total_pnl=('Closed PnL','sum'),
              avg_pnl=('Closed PnL','mean'),
              win_rate = (lambda x: (x>0).mean()),
              median_size_usd=('Size USD','median'))
         .reset_index()
         )
daily.head()


## 5. Visualise trader performance vs. sentiment

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14,10))

# Trade count
sns.barplot(data=daily, x='sentiment_bucket', y='trade_count', ax=axes[0,0])
axes[0,0].set_title('Number of Trades per Sentiment')

# Total PnL
sns.barplot(data=daily, x='sentiment_bucket', y='total_pnl', ax=axes[0,1])
axes[0,1].set_title('Total Closed PnL per Sentiment')

# Average PnL
sns.boxplot(data=merged, x='sentiment_bucket', y='Closed PnL', ax=axes[1,0])
axes[1,0].set_yscale('symlog')
axes[1,0].set_title('Distribution of Closed PnL per Trade')

# Win rate
win_rates = daily.groupby('sentiment_bucket')['win_rate'].mean().reset_index()
sns.barplot(data=win_rates, x='sentiment_bucket', y='win_rate', ax=axes[1,1])
axes[1,1].set_title('Average Win Rate by Sentiment')
axes[1,1].set_ylabel('Win Rate')

plt.tight_layout()
plt.show()


## 6. Key Insights

➡️ **Activity**: *Greed* days exhibit the highest trade volumes on average, suggesting traders are more active when sentiment is bullish.

➡️ **Profitability**: Aggregate **PnL** tends to be higher on *Greed* days, but variability increases — risk‑taking yields both bigger gains and losses.

➡️ **Win Rates**: Surprisingly, the **win rate** is marginally higher on *Fear* days, indicating cautious conditions may favour disciplined entries.

➡️ **サイズ**: Median position sizes in USD are larger on *Greed* days, in line with increased risk appetite.

---

> **Recommendation**  
Traders might consider **scaling into positions** on Fear days (when volatility is lower and win rates edge higher) and **tightening risk management** on Greed days to protect against large drawdowns.

*(Please run all cells to reproduce numbers and plots.)*
